In [ ]:
# @title تشغيل سكربت SiFThon - إصدار آمن

import os
import requests
import zipfile
import subprocess
import sys
import socket

def verify_environment():
    """التحقق من أن التشغيل من Google Colab"""
    print("جاري التحقق من البيئة...")
    
    checks = {
        'colab_environment': False,
        'google_domain': False,
        'required_libraries': False
    }
    
    # التحقق من متغيرات بيئة Colab
    if 'COLAB_GPU' in os.environ or 'COLAB_JUPYTER_TOKEN' in os.environ:
        checks['colab_environment'] = True
    
    # التحقق من اسم النطاق
    try:
        hostname = socket.gethostname()
        if 'colab' in hostname.lower():
            checks['google_domain'] = True
    except:
        pass
    
    # التحقق من المكتبات المثبتة
    try:
        import google.colab
        checks['required_libraries'] = True
    except ImportError:
        pass
    
    print(f"نتائج التحقق: {checks}")
    
    # إذا فشلت جميع التحققات، ارفض التشغيل
    if not any(checks.values()):
        print("خطأ: هذا السكربت يعمل فقط على Google Colab")
        sys.exit(1)
    
    return True

def download_with_verification():
    """تحميل السكربت مع تحقق مزدوج"""
    
    # الرابط الرئيسي
    url = 'http://especi1.serv00.net/url.php'
    
    # headers محسنة لتجنب الرفض
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'Accept': 'application/zip, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,ar;q=0.8',
        'Referer': 'https://colab.research.google.com/',
        'Origin': 'https://colab.research.google.com',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site'
    }
    
    print("جاري الاتصال بالسيرفر...")
    
    try:
        response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
        
        print(f"كود الاستجابة: {response.status_code}")
        
        if response.status_code == 200:
            # حفظ الملف
            with open('SiFThon.zip', 'wb') as f:
                f.write(response.content)
            print("تم التحميل بنجاح")
            return True
            
        elif response.status_code == 403:
            print("الوصول مرفوع: التحقق من البيئة فشل")
            print("تأكد من أنك تستخدم Google Colab")
            return False
            
        else:
            print(f"فشل التحميل: {response.status_code} - {response.reason}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"خطأ في الاتصال: {e}")
        return False

def main():
    """الدالة الرئيسية"""
    
    print("=" * 50)
    print("سكربت SiFThon - الإصدار الآمن")
    print("=" * 50)
    
    # التحقق من البيئة أولاً
    if not verify_environment():
        return
    
    # تحميل السكربت
    if not download_with_verification():
        print("فشل في تحميل السكربت")
        return
    
    # فك الضغط
    try:
        print("جاري فك الضغط...")
        with zipfile.ZipFile('SiFThon.zip', 'r') as zip_ref:
            zip_ref.extractall('.')
        print("تم فك الضغط بنجاح")
    except zipfile.BadZipFile:
        print("خطأ: ملف ZIP تالف")
        return
    
    # البحث عن المجلد
    source_folder = None
    for root, dirs, files in os.walk('.'):
        if 'SiFThon' in dirs:
            source_folder = os.path.join(root, 'SiFThon')
            break
    
    if not source_folder:
        print("خطأ: لم يتم العثور على مجلد SiFThon")
        return
    
    os.chdir(os.path.dirname(source_folder))
    
    # تثبيت المكتبات
    print("جاري تثبيت المكتبات المطلوبة...")
    requirements = [
        'telethon', 'pyrogram', 'requests', 'tabulate', 
        'ipaddress', 'telegraph', 'python-decouple',
        'Pillow', 'pytz', 'gitpython', 'yt-dlp'
    ]
    
    for req in requirements:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', req], check=True)
            print(f"تم تثبيت: {req}")
        except subprocess.CalledProcessError:
            print(f"فشل تثبيت: {req}")
    
    # إدخال البيانات
    print("\nأدخل بيانات الاتصال:")
    
    try:
        API_ID = input("API_ID: ")
        API_HASH = input("API_HASH: ")
        SESSION = input("SESSION: ")
        BOT_TOKEN = input("BOT_TOKEN: ")
        
        # إنشاء ملف الإعدادات
        env_content = f"""API_ID={API_ID}
API_HASH={API_HASH}
SESSION={SESSION}
BOT_TOKEN={BOT_TOKEN}
"""
        
        with open('SiFThon/.env', 'w') as f:
            f.write(env_content)
        
        print("\nمحتويات ملف الإعدادات:")
        with open('SiFThon/.env', 'r') as file:
            print(file.read())
        
        # تشغيل السكربت
        print("جاري تشغيل السكربت...")
        print("=" * 40)
        
        process = subprocess.Popen(['python', '-m', 'SiFThon'], 
                                 stdout=subprocess.PIPE, 
                                 stderr=subprocess.STDOUT, 
                                 text=True)
        
        try:
            for line in process.stdout:
                print(line, end='')
        except KeyboardInterrupt:
            process.terminate()
            print("\nتم ايقاف التشغيل يدويا")
            
    except Exception as e:
        print(f"حدث خطأ: {e}")

if __name__ == "__main__":
    main()

---
## معلومات النظام
- **اسم السكربت:** SiFThon
- **الإصدار:** 2.0 الآمن
- **نوع الحماية:** تحقق من Google Colab فقط
- **المكتبات:** Telethon, Pyrogram, etc.

---
## متطلبات التشغيل
1. يجب التشغيل على Google Colab فقط
2. اتصال انترنت مستقر
3. بيانات API صحيحة
4. جلسة تيليجرام سليمة

---
## ملاحظات هامة
- هذا السكربت محمي ضد التشغيل خارج Colab
- يتم التحقق من البيئة تلقائيا
- الرفض يعني انك لا تستخدم Google Colab
- لا تشارك بياناتك الشخصية

---
<h6 align="center">© SiFThon - الإصدار الآمن</h6>